<a href="https://colab.research.google.com/github/RenanNB360/FineTuningLLM-com-LoRA-para-Analise-de-Sentimentos-em-Texto/blob/main/FineTuningLLM_com_LoRA_para_An%C3%A1lise_de_Sentimentos_em_Texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalando e Carregando Pacotes

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0

In [ ]:
!pip install -q trl==0.4.7 gradio==3.37.0 protobuf==3.20.3 scipy==1.11.1

In [ ]:
!pip install -q sentencepiece==0.1.99 tokenizers==0.13.3 datasets==2.16.1

In [ ]:
import os
import torch
import datasets
import pandas as pd
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          TrainingArguments,
                          pipeline,
                          logging)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
logging.set_verbosity(logging.CRITICAL)

In [ ]:
if torch.cuda.is_available():
    print('Número de GPUs:', torch.cuda.device_count())
    print('Modelo GPU:', torch.cuda.get_device_name(0))
    print('Total Memória [GB] da GPU:',torch.cuda.get_device_properties(0).total_memory / 1e9)

Número de GPUs: 1
Modelo GPU: NVIDIA A100-SXM4-40GB
Total Memória [GB] da GPU: 42.481811456


In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
nome_dataset = "dataset.csv"

In [ ]:
dataset_carregado = load_dataset('csv', data_files = nome_dataset, delimiter = ',')

In [ ]:
dataset_carregado

DatasetDict({
    train: Dataset({
        features: ['train'],
        num_rows: 17057
    })
})

In [ ]:
repositorio_hf = "NousResearch/Llama-2-7b-chat-hf"

In [ ]:
modelo = "novo_modelo"

## Definindo os Parâmetros de Configuração

In [ ]:
lora_r = 32
lora_alpha = 16
lora_dropout = 0.1

In [ ]:
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

In [ ]:
output_dir = "saida"
num_train_epochs = 1
fp16 = True
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03

In [ ]:
group_by_length = True
save_steps = 0
logging_steps = 400

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [ ]:
bnb_config = BitsAndBytesConfig(load_in_4bit = use_4bit,
                                bnb_4bit_quant_type = bnb_4bit_quant_type,
                                bnb_4bit_compute_dtype = compute_dtype,
                                bnb_4bit_use_double_quant = use_nested_quant)

In [ ]:
modelo = AutoModelForCausalLM.from_pretrained(repositorio_hf,
                                              quantization_config = bnb_config,
                                              device_map = "auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
modelo.config.use_cache = False
modelo.config.pretraining_tp = 1

In [ ]:
tokenizador = AutoTokenizer.from_pretrained(repositorio_hf, trust_remote_code = True)
tokenizador.pad_token = tokenizador.eos_token
tokenizador.padding_side = "right"

In [ ]:
peft_config = LoraConfig(lora_alpha = lora_alpha,
                         lora_dropout = lora_dropout,
                         r = lora_r,
                         bias = "none",
                         task_type = "CAUSAL_LM")

In [ ]:
training_arguments = TrainingArguments(output_dir = output_dir,
                                       num_train_epochs = num_train_epochs,
                                       per_device_train_batch_size = per_device_train_batch_size,
                                       gradient_accumulation_steps = gradient_accumulation_steps,
                                       optim = optim,
                                       save_steps = save_steps,
                                       logging_steps = logging_steps,
                                       learning_rate = learning_rate,
                                       weight_decay = weight_decay,
                                       fp16 = fp16,
                                       bf16 = bf16,
                                       max_grad_norm = max_grad_norm,
                                       max_steps = max_steps,
                                       warmup_ratio = warmup_ratio,
                                       group_by_length = group_by_length,
                                       lr_scheduler_type = lr_scheduler_type)

In [ ]:
trainer = SFTTrainer(model = modelo,
                         train_dataset = dataset_carregado['train'],
                         peft_config = peft_config,
                         dataset_text_field = "train",
                         max_seq_length = None,
                         tokenizer = tokenizador,
                         args = training_arguments,
                         packing = False)

> Treinamento do Modelo com o Ajuste Fino

In [ ]:
%%time
dsa_trainer.train()

{'loss': 2.509, 'learning_rate': 0.000197905384816968, 'epoch': 0.09}
{'loss': 2.2269, 'learning_rate': 0.00018733342277442523, 'epoch': 0.19}
{'loss': 2.1996, 'learning_rate': 0.00016876519303924464, 'epoch': 0.28}
{'loss': 2.1828, 'learning_rate': 0.00014390080687266013, 'epoch': 0.38}
{'loss': 2.2035, 'learning_rate': 0.0001150168530248306, 'epoch': 0.47}
{'loss': 2.2051, 'learning_rate': 8.483300876675342e-05, 'epoch': 0.56}
{'loss': 2.1742, 'learning_rate': 5.603097596449888e-05, 'epoch': 0.66}
{'loss': 2.1505, 'learning_rate': 3.1179692069489296e-05, 'epoch': 0.75}
{'loss': 2.1612, 'learning_rate': 1.2629610937636283e-05, 'epoch': 0.84}
{'loss': 2.1641, 'learning_rate': 2.0791821400001245e-06, 'epoch': 0.94}
{'train_runtime': 1815.4954, 'train_samples_per_second': 9.395, 'train_steps_per_second': 2.349, 'train_loss': 2.2141866310540004, 'epoch': 1.0}
CPU times: user 28min 28s, sys: 1min 53s, total: 30min 21s
Wall time: 30min 17s


TrainOutput(global_step=4265, training_loss=2.2141866310540004, metrics={'train_runtime': 1815.4954, 'train_samples_per_second': 9.395, 'train_steps_per_second': 2.349, 'train_loss': 2.2141866310540004, 'epoch': 1.0})

In [ ]:
trainer.model.save_pretrained(modelo)

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

In [ ]:
prompt = "It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement"

In [ ]:
pipe = pipeline(task = "text-generation",
                model = modelo,
                tokenizer = tokenizador,
                max_length = 200)

In [ ]:
resultado = pipe(f"<s>[INST] {prompt} [/INST]")

In [ ]:
print(resultado)

[{'generated_text': "<s>[INST] It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement [/INST] Positive [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST"}]


In [ ]:
print(resultado[0]['generated_text'])

<s>[INST] It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement [/INST] Positive [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST] Negative [/INST


In [ ]:
del modelo
del pipe
del trainer
import gc
gc.collect()

0

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(repositorio_hf,
                                                  low_cpu_mem_usage = True,
                                                  return_dict = True,
                                                  torch_dtype = torch.float16,
                                                  device_map = "auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
modelo_final = PeftModel.from_pretrained(base_model, modelo)

In [ ]:
modelo_final = modelo_final.merge_and_unload()

In [ ]:
tokenizador = AutoTokenizer.from_pretrained(repositorio_hf, trust_remote_code = True)
tokenizador.pad_token = tokenizador.eos_token
tokenizador.padding_side = "right"

In [ ]:
modelo_final.save_pretrained('novo_modelo-dsa-llm-projeto2')
tokenizador.save_pretrained('novo_modelo-dsa-llm-projeto2')

('novo_modelo-dsa-llm-projeto2/tokenizer_config.json',
 'novo_modelo-dsa-llm-projeto2/special_tokens_map.json',
 'novo_modelo-dsa-llm-projeto2/tokenizer.model',
 'novo_modelo-dsa-llm-projeto2/added_tokens.json',
 'novo_modelo-dsa-llm-projeto2/tokenizer.json')

In [ ]:
prompt = "It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement"

In [ ]:
pipe = pipeline(task = "text-generation",
                model = modelo_final,
                tokenizer = tokenizador,
                max_length = 200)

In [ ]:
resultado = pipe(f"<s>[INST] {prompt} [/INST]")

In [ ]:
print(resultado)

[{'generated_text': "<s>[INST] It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement [/INST] Positive:\n  * The movie is visually stunning, with a unique and captivating style.\n  * The story is engaging and thought-provoking, with a strong emotional impact.\n  * The acting is superb, with a standout performance from Joaquin Phoenix.\n  * The movie is well-written and well-directed, with a strong sense of pacing and tone.\n  * The movie is a powerful and moving experience, with a strong emotional impact.\n  * The movie is a must-see for fans of the genre, and for anyone who appreciates a well-crafted film.\n  * The movie is a great example of the power of cinema, and the ability of filmmakers to create"}]


In [ ]:
print(resultado[0]['generated_text'])

<s>[INST] It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement [/INST] Positive:
  * The movie is visually stunning, with a unique and captivating style.
  * The story is engaging and thought-provoking, with a strong emotional impact.
  * The acting is superb, with a standout performance from Joaquin Phoenix.
  * The movie is well-written and well-directed, with a strong sense of pacing and tone.
  * The movie is a powerful and moving experience, with a strong emotional impact.
  * The movie is a must-see for fans of the genre, and for anyone who appreciates a well-crafted film.
  * The movie is a great example of the power of cinema, and the ability of filmmakers to create


In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()